using a kdtree to find the line closest to each point
chose this method, moved to coastlines.py

In [25]:
import geopandas as gpd
import pandas as pd

sites = pd.read_csv(r"C:\Users\CAMG038492\Code\Climatology\archive\locations.csv")
file = r"C:\Users\CAMG038492\WSP O365\WDS-Digital Environment - Climatology Datasets - Climatology Datasets\Natural Earth Coastline Data\ne_110m_coastline\ne_110m_coastline.shp"
coastline = gpd.read_file(file)
coastline

,scalerank,featurecla,min_zoom,geometry
0,1,Coastline,1.0,"LINESTRING (-163.7129 -78.59567, -163.1058 -78..."
1,0,Coastline,0.0,"LINESTRING (-6.19788 53.86757, -6.03299 53.153..."
2,0,Coastline,0.0,"LINESTRING (141.00021 -2.60015, 142.73525 -3.2..."
3,0,Coastline,0.0,"LINESTRING (114.20402 4.52587, 114.59996 4.900..."
4,1,Coastline,1.5,"LINESTRING (-93.61276 74.98, -94.15691 74.5923..."
...,...,...,...,...
129,0,Coastline,0.0,"LINESTRING (93.77766 81.0246, 95.94089 81.2504..."
130,0,Coastline,0.0,"LINESTRING (-96.01643 80.60232, -95.32345 80.9..."
131,0,Coastline,0.0,"LINESTRING (-91.58702 81.89429, -90.1 82.085, ..."
132,0,Coastline,0.0,"LINESTRING (-46.76379 82.62796, -43.40644 83.2..."


In [26]:
from shapely.geometry import Point
import numpy as np
from scipy.spatial import cKDTree

points = []
for geom in coastline.geometry:
    if geom.geom_type == 'LineString':
        points.extend(list(geom.coords))
    elif geom.geom_type == 'MultiLineString':
        for line in geom:
            points.extend(list(line.coords))

coords = np.array(points)
tree = cKDTree(coords)
tree

In [27]:
from geopy.distance import geodesic

def nearest(lat, lon, tree, coords):
    i, idx = tree.query([lon, lat])
    nearestpt = coords[idx]
    return geodesic((lat, lon), (nearestpt[1], nearestpt[0])).kilometers

def v_nearest(row):
    return nearest(lat = row.Y, lon = row.X, tree = tree, coords = coords)

distance = nearest(45.4215, -75.6972, tree, coords)
distance

387.5212781629498

In [28]:
from math import radians, sin, cos

def angles(lat, lon, maxkm=50, step=1, tree=None, coords=None):
    total_angles = int(360 / step)
    hit_count = 0

    for angle in range(0, 360, step):
        rad = radians(angle)

        end_lat = lat + (maxkm / 111) * cos(rad)
        end_lon = lon + (maxkm / (111 * cos(radians(lat)))) * sin(rad)

        i, idx = tree.query([end_lon, end_lat])
        nearest_point = coords[idx]

        actual_distance = geodesic((lat, lon), (nearest_point[1], nearest_point[0])).kilometers

        if actual_distance <= maxkm:
            hit_count += 1

    return (hit_count / total_angles) * 100

def v_angles(row, km):
    return angles(lat = row.Y, lon = row.X, maxkm = km, tree = tree, coords = coords)

coverage = angles(45.4215, -75.6972, maxkm=1000, step=1, tree=tree, coords=coords)
coverage


80.55555555555556

In [29]:
sites

,X,Y,Z,File Name,NAME,Land
0,-63.573566,44.646244,0,testing.csv,spot1,NaN
1,-59.304967,47.615202,0,gulf.csv,real gulf point,NaN
2,-57.152780,48.111857,0,nl.csv,real nl point,NaN
3,-59.955059,45.929556,-,ns.csv,real ns point,NaN
4,-75.703000,45.423000,0,testing.csv,parliament for fun,NaN
5,-59.991996,46.937680,0,gulf.csv,real gulf point over the water,NaN
6,0.000000,0.000000,0,testing,rot pol test 1,NaN
7,10.000000,10.000000,0,testing,rot pol test 2,NaN


In [30]:
sites["Nearest coast (km)"] = sites.apply(v_nearest, axis = 1)
sites

,X,Y,Z,File Name,NAME,Land,Nearest coast (km)
0,-63.573566,44.646244,0,testing.csv,spot1,NaN,25.428667
1,-59.304967,47.615202,0,gulf.csv,real gulf point,NaN,3.211549
2,-57.152780,48.111857,0,nl.csv,real nl point,NaN,61.309723
3,-59.955059,45.929556,-,ns.csv,real ns point,NaN,11.847872
4,-75.703000,45.423000,0,testing.csv,parliament for fun,NaN,387.860461
5,-59.991996,46.937680,0,gulf.csv,real gulf point over the water,NaN,40.790887
6,0.000000,0.000000,0,testing,rot pol test 1,NaN,564.827571
7,10.000000,10.000000,0,testing,rot pol test 2,NaN,601.407265


In [31]:
sites["Angular coverage of coastline at 50km radius (%)"] = sites.apply(v_angles, km=50, axis=1)
sites

,X,Y,Z,File Name,NAME,Land,Nearest coast (km),Angular coverage of coastline at 50km radius (%)
0,-63.573566,44.646244,0,testing.csv,spot1,NaN,25.428667,57.500000
1,-59.304967,47.615202,0,gulf.csv,real gulf point,NaN,3.211549,81.111111
2,-57.152780,48.111857,0,nl.csv,real nl point,NaN,61.309723,0.000000
3,-59.955059,45.929556,-,ns.csv,real ns point,NaN,11.847872,62.500000
4,-75.703000,45.423000,0,testing.csv,parliament for fun,NaN,387.860461,0.000000
5,-59.991996,46.937680,0,gulf.csv,real gulf point over the water,NaN,40.790887,37.777778
6,0.000000,0.000000,0,testing,rot pol test 1,NaN,564.827571,0.000000
7,10.000000,10.000000,0,testing,rot pol test 2,NaN,601.407265,0.000000
